Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1862]:
import heapq
import json
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [1863]:
PUZZLE_DIM = 10
RANDOMIZE_STEPS = 100_000
FINAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

action = namedtuple('Action', ['pos1', 'pos2'])

## Helper functions

In [1864]:
def available_actions(state: np.ndarray, layer=0, preferencial_dir=None) -> list["Action"]:
    x, y = [int(_[0]) for _ in np.where(state == 0)]  
    actions = []  
    
    # move definition
    if x > layer :  # up
        actions.append(action((x, y), (x - 1, y)))
    if y < PUZZLE_DIM - 1:  # right
        actions.append(action((x, y), (x, y + 1)))
    if y > 0:  # left
        actions.append(action((x, y), (x, y - 1)))
    if x < PUZZLE_DIM - 1:  # down
        actions.append(action((x, y), (x + 1, y)))
    
    # preferencial directions
    direction_order = {
        "up": ["up", "right", "left", "down"],
        "down": ["down", "right", "left", "up"],
        "right": ["right", "down", "up", "left"],
        "left": ["left", "down", "up", "right"],
        None: ["left","up","right", "down"], 
    }
    
    # Map actions and strings if they are present
    action_map = {
        "up": action((x, y), (x - 1, y)) if x > layer else None,
        "right": action((x, y), (x, y + 1)) if y < PUZZLE_DIM - 1 else None,
        "left": action((x, y), (x, y - 1)) if y > 0 else None,
        "down": action((x, y), (x + 1, y)) if x < PUZZLE_DIM - 1 else None,
    }
    
    # Ordina le azioni disponibili secondo il preferencial_dir
    sorted_actions = [
        action_map[dir] for dir in direction_order[preferencial_dir] if action_map[dir] is not None
    ]
    
    return sorted_actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_final_sol(state: np.ndarray) -> bool:
    return np.array_equal(state, FINAL_STATE)

## Random Initialization

In [1865]:
curr_state = FINAL_STATE
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    curr_state = do_action(curr_state, choice(available_actions(curr_state)))
curr_state

Randomizing: 100%|██████████| 100000/100000 [00:01<00:00, 61172.24it/s]


array([[52,  1, 98, 57, 95,  2, 55, 43, 22, 91],
       [33, 27, 48, 73, 28,  8,  4, 14,  6, 88],
       [49, 86, 69, 89,  9, 34, 31, 97, 42, 38],
       [77, 11, 51, 79, 46, 62, 60, 17, 59, 26],
       [37, 29, 39, 23,  5, 13, 25, 71, 66, 96],
       [90, 40, 41, 16, 45, 61, 70, 20, 94, 56],
       [78, 24, 68, 93, 63, 47, 83, 50, 92, 30],
       [99, 64, 58, 82, 85,  3, 32,  7, 74, 75],
       [35, 36, 10, 87, 76, 21, 19, 15, 18, 84],
       [80, 67, 53, 81, 54, 44, 12, 72, 65,  0]])

## Move Functions

In [1866]:
def distance_between(x1,y1,x2,y2):
    return  np.abs(x1 - x2) + np.abs(y1 - y2)

In [1867]:
def actions_from_zero_to_point(state:np.ndarray,x,y,layer):
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    last_two_flag = False
    if layer == len(FINAL_STATE)-2:
        last_two_flag=True
    while distance_between(x,y,x0,y0) > 1:
        for a in available_actions(current,layer,"right"):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if distance_between(x,y,nx0,ny0)< distance_between(x,y,x0,y0) and (current[nx0,ny0] > n or last_two_flag):
                current = next
                path.append(current)
                print(current)
                x0, y0 = nx0, ny0
                break
    return path, current

In [1868]:
def put_zero_below(state:np.ndarray, x, y,layer, dir, couple=None):
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    
    if couple != None and current[x0,y0-1] == couple:
        a = action((x0, y0), (x0-1, y0)) #up
        current = do_action(current,a)
        path.append(current)
        x0, y0 = np.where(current==0)

    while distance_between(x+1,y,x0,y0) != 0:
        for a in available_actions(current,layer,dir):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if next[x,y]==n and distance_between(x+1,y,nx0,ny0)< distance_between(x+1,y,x0,y0):
                current = next
                path.append(current)
                print("here")
                print(current)
                x0, y0 = nx0, ny0
                break
    return path, current

In [1869]:
def slide_row(state:np.ndarray,layer):
    current = state
    path = []
    x0, y0 = np.where(current==0)
    print(f"dest : {x0},{y0}")
    d_x, d_y = x0 -1 , len(FINAL_STATE)-1 
    print(f"dest : {d_x},{d_y}")
    while distance_between(d_x,d_y,x0,y0) != 0:
        for a in available_actions(current,layer,"up"):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if distance_between(d_x,d_y,nx0,ny0)< distance_between(d_x,d_y,x0,y0):
                current = next
                path.append(current)
                print(current)
                x0, y0 = nx0, ny0
                break
    return path, current
    

In [1870]:
def slide_row_back(state:np.ndarray,layer):
    current = state
    path = []
    x0, y0 = np.where(current==0)
    d_x, d_y = x0 + 1 , 0
    while distance_between(d_x,d_y,x0,y0) != 0:
        for a in available_actions(current,layer,"left"):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if distance_between(d_x,d_y,nx0,ny0)< distance_between(d_x,d_y,x0,y0):
                current = next
                path.append(current)
                print(current)
                x0, y0 = nx0, ny0
                break
    return path, current

In [1871]:
def put_last_right(state:np.ndarray):
    current = state
    path = []
    x,y = np.where(current==0)
    move_set= [
        action((x, y), (x + 1, y)), #down
        action((x + 1, y), (x + 1, y - 1)), #left
        action((x + 1, y - 1), (x, y - 1)) #up
    ]
    for act in move_set:
        current = do_action(current,act)
        print(current)
        path.append(current)
        
    return path, current

In [1872]:
def action_move_right_col(state:np.ndarray,x,y,layer,x_f,y_f):
    '''moves a number to the correct y position'''
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    prev_x0,prev_y0 = x0,y0
    while y != y_f:
        if y<y_f:
            # move n right
            while (y0,x0)!=(y+1,x):
                same_line = x0==x
                for a in available_actions(current,layer,"right"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and np.abs(y-ny0) < 2 and np.abs(x-nx0) < 2 and (nx0,ny0)!=(prev_x0,prev_y0):
                        if ( ( same_line and np.abs(ny0-(y+1))<=np.abs(y0-(y+1)) ) or ( not same_line and distance_between(nx0,ny0,x,y+1)<distance_between(x0,y0,x,y+1) ) ):
                            prev_x0, prev_y0 = x0,y0
                            current = next
                            path.append(current)
                            print(current)

                            x0, y0 = nx0, ny0
                            break
            left = action((x0, y0), (x0, y0 - 1))
            current = do_action(current,left)
            path.append(current)
            print(current)

            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
        else:
            # move n left
            while (y0,x0)!=(y-1,x):
                same_line = x0==x
                for a in available_actions(current,layer,"left"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and np.abs(y-ny0) < 2 and np.abs(x-nx0) < 2 and (nx0,ny0)!=(prev_x0,prev_y0):
                        if ( ( same_line and np.abs(ny0-(y-1))<=np.abs(y0-(y-1)) ) or ( not same_line and distance_between(nx0,ny0,x,y-1)<distance_between(x0,y0,x,y-1) ) ):
                            prev_x0, prev_y0 = x0,y0
                            current = next
                            path.append(current)
                            print(current)

                            x0, y0 = nx0, ny0
                            break
            right = action((x0, y0), (x0, y0 + 1))
            current = do_action(current,right)
            print(current)

            path.append(current)
            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
    return path, current

In [1873]:
def action_move_right_row(state:np.ndarray,x,y,layer,x_f,y_f):
    '''moves a number to the correct x position'''
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    
    prev_x0,prev_y0 = x0,y0
    while x != x_f:
        if x<x_f:
            # move n down
            while (y0,x0)!=(y,x+1):
                same_col = y0==y
                for a in available_actions(current,layer, "down"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and distance_between(nx0,ny0,x+1,y)<distance_between(x0,y0,x+1,y):
                        prev_x0, prev_y0 = x0,y0
                        current = next
                        print(current)
                        print("moving down")
                        path.append(current)
                        x0, y0 = nx0, ny0
                        break
            up = action((x0, y0), (x0 - 1, y0))
            current = do_action(current,up)
            path.append(current)
            print(current)
            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
        else:
            # move n up
            if ((x,y)==(x_f+1,y_f) and (x0,y0)!=(x_f,y_f) and n>1 and y0 < y and n != (layer*len(FINAL_STATE))+len(FINAL_STATE)):
                moves, current = put_zero_below(current,x,y,layer,"right")
                path.extend(moves)
                x,y = np.where(current==n)
                x0, y0 = np.where(current==0)
            while (y0,x0)!=(y,x-1):
                same_col = y0==y
                for a in available_actions(current,layer, "up"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and np.abs(y-ny0) < 2 and np.abs(x-nx0) < 2 and (nx0,ny0)!=(prev_x0,prev_y0) :
                        if ( ( same_col and np.abs(nx0-(x-1)) <= np.abs(x0-(x-1)) ) or ( not same_col and distance_between(nx0,ny0,x-1,y)<distance_between(x0,y0,x-1,y)  ) ):
                            prev_x0, prev_y0 = x0,y0
                            current = next
                            path.append(current)
                            print(current)
                            x0, y0 = nx0, ny0
                            break
            down = action((x0, y0), (x0 + 1, y0))
            current = do_action(current,down)
            path.append(current)
            print(current)
            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
    return path, current

In [1874]:
from dataclasses import dataclass

@dataclass
class Story_Item:
    score: int
    pattern : list

In [1875]:
def manhattan_distance(state: np.ndarray) -> int:
    """Calcola la distanza di Manhattan per tutte le tessere."""
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  # Ignora il vuoto
                target_x, target_y = np.where(FINAL_STATE==value)
                distance += abs(x - target_x) + abs(y - target_y)
    return distance

In [1876]:
def manhattan_distance_point(state: np.ndarray, x, y) -> int:
    """Calcola la distanza di Manhattan per un punto."""
    distance = 0
    target_x, target_y = np.where(FINAL_STATE==state[x,y])
    distance += abs(x - target_x) + abs(y - target_y)
    return distance

In [1877]:
def lead_n_to_pos(state:np.ndarray,n:int,layer,x_f,y_f):
    current = state
    path = []
    x , y = np.where(current==n)
    x0, y0 = np.where(current==0)
    
    # Move zero near to number to move
    if ((x,y)!=(x_f,y_f)) and (np.abs(x-x0)>1 or np.abs(y-y0)>1):
        steps, current = actions_from_zero_to_point(current,x,y,layer)
        path.extend(steps)
        x , y = np.where(current==n)
    # move the number to right col
    steps, current = action_move_right_col(current,x,y,layer,x_f,y_f)
    path.extend(steps)
    x , y = np.where(current==n)
    # move number to right row
    steps, current = action_move_right_row(current,x,y,layer,x_f,y_f)
    path.extend(steps)
    #print(path[-1])
    return path, current


In [1878]:
#lead_n_to_pos(curr_state,1)

## Layer Solving

In [1879]:
def solve_layer(state, layer):
    current = state
    path = []
    path.append(current)
    # solve first n-1 number in the line
    for n in range((layer*len(FINAL_STATE))+1,(layer*len(FINAL_STATE))+len(FINAL_STATE)+1):
        x,y = np.where(current==n)
        x_f,y_f = np.where(FINAL_STATE==n)
        x0 , y0 = np.where(current==0)
        if ((x,y)!=(x_f,y_f)):
            if y_f == len(FINAL_STATE)-1 and ((x0,y0)!=(x_f,y_f) or (x,y)!=(x_f+1,y_f)):
                print("x_f up")
                x_f +=1
            moves_done, current = lead_n_to_pos(current,n,layer,x_f,y_f)
            print("moved")
            path.extend(moves_done)
        
    if(current[layer,len(FINAL_STATE)-1]==FINAL_STATE[layer,len(FINAL_STATE)-1]): 
        return path,current    
    # move zero below the first number of the layer to be able to slide the row
    x1, y1 = np.where(current==(layer*len(FINAL_STATE))+1)
    print(f"under {(layer*len(FINAL_STATE))+1}")
    moves_done, current = put_zero_below(current,x1,y1,layer+1,"left")
    path.extend(moves_done)

    #slide the row in order to leave space for the last number
    moves_done, current = slide_row(current,layer)
    path.extend(moves_done)

    # put last number right and zero to the left
    moves_done, current = put_last_right(current)
    path.extend(moves_done)

    #slide the row in order to leave space for the last number
    moves_done, current = slide_row_back(current,layer)
    path.extend(moves_done)

    return path,current

In [1880]:
path = []
init_state = curr_state
for layer in range(len(FINAL_STATE)-2):
    print(f"Solving Layer N: {layer}")
    moves,init_state = solve_layer(init_state, layer)
    print(f"N moves layer {layer}: {len(moves)}")
    moves[-1]
    path.extend(moves)

Solving Layer N: 0
[[52  1 98 57 95  2 55 43 22 91]
 [33 27 48 73 28  8  4 14  6 88]
 [49 86 69 89  9 34 31 97 42 38]
 [77 11 51 79 46 62 60 17 59 26]
 [37 29 39 23  5 13 25 71 66 96]
 [90 40 41 16 45 61 70 20 94 56]
 [78 24 68 93 63 47 83 50 92 30]
 [99 64 58 82 85  3 32  7 74 75]
 [35 36 10 87 76 21 19 15 18  0]
 [80 67 53 81 54 44 12 72 65 84]]
[[52  1 98 57 95  2 55 43 22 91]
 [33 27 48 73 28  8  4 14  6 88]
 [49 86 69 89  9 34 31 97 42 38]
 [77 11 51 79 46 62 60 17 59 26]
 [37 29 39 23  5 13 25 71 66 96]
 [90 40 41 16 45 61 70 20 94 56]
 [78 24 68 93 63 47 83 50 92 30]
 [99 64 58 82 85  3 32  7 74  0]
 [35 36 10 87 76 21 19 15 18 75]
 [80 67 53 81 54 44 12 72 65 84]]
[[52  1 98 57 95  2 55 43 22 91]
 [33 27 48 73 28  8  4 14  6 88]
 [49 86 69 89  9 34 31 97 42 38]
 [77 11 51 79 46 62 60 17 59 26]
 [37 29 39 23  5 13 25 71 66 96]
 [90 40 41 16 45 61 70 20 94 56]
 [78 24 68 93 63 47 83 50 92  0]
 [99 64 58 82 85  3 32  7 74 30]
 [35 36 10 87 76 21 19 15 18 75]
 [80 67 53 81 54 44 12

In [1881]:
len(path)

2726

In [1882]:
path[-1]

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
       [31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
       [41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
       [51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
       [61, 62, 63, 64, 65, 66, 67, 68, 69, 70],
       [71, 72, 73, 74, 75, 76, 77, 78, 79, 80],
       [ 0, 99, 87, 96, 81, 89, 86, 85, 93, 97],
       [83, 95, 82, 98, 90, 92, 91, 88, 84, 94]])

In [1883]:
def put_cuple_near(state):
    path=[]
    current = state
    x0,y0 = np.where(current==0)
    moves = [
        action((x0, y0), (x0-1, y0)), # up
        action((x0-1, y0), (x0-1, y0 +1)), # right
        action((x0-1, y0+1), (x0, y0+1)), # down
        action((x0, y0+1), (x0, y0 +2)) # right
    ]
    for a in moves:
        current = do_action(current,a)
        path.append(current)
        print(current)
    return path,current
    

In [1884]:
def put_cuple_right(state):
    path=[]
    current = state
    x0,y0 = np.where(current==0)
    moves = [
        action((x0, y0), (x0-1, y0)), # up
        action((x0-1, y0), (x0-1, y0 -1)), # left
        action((x0-1, y0-1), (x0-1, y0-2)), # left
        action((x0-1, y0-2), (x0, y0 -2)), # down
        action((x0, y0-2), (x0, y0 -1)) # right
    ]
    for a in moves:
        current = do_action(current,a)
        path.append(current)
        print(current)
    return path,current

In [1885]:
def solve_last_two_layers(state:np.ndarray):
    ''' Solve the last two layer together solving column per column at couple'''
    path=[]
    current=state
    layer = len(FINAL_STATE)-2

    for col in range(PUZZLE_DIM-1):
        n_up = len(FINAL_STATE)*(len(FINAL_STATE)-2)+col+1
        n_down = len(FINAL_STATE)*(len(FINAL_STATE)-1)+col+1

        x_up_f, y_up_f = np.where(FINAL_STATE==n_up)
        x_down_f, y_down_f = np.where(FINAL_STATE==n_down)
        #First lead n_up to right pos
        moves,current = lead_n_to_pos(current,n_up,layer,x_up_f, y_up_f)
        path.extend(moves)
        print(n_up,n_down)

        # If n_down can be placed we place it
        if current[len(FINAL_STATE)-1,col] == 0 and current[len(FINAL_STATE)-1,col+1] == n_down:
            moves,current = lead_n_to_pos(current,n_down,layer,x_down_f,y_down_f)
            path.extend(moves)
            continue

        # if n_down is in the right place we go on, otherwise we put it two columns away from the right position
        if current[len(FINAL_STATE)-1,col] != n_down:
            moves,current = lead_n_to_pos(current,n_down,layer,x_down_f,y_down_f+2)
            path.extend(moves)
        else:
            
            continue

        
        # Put zero below n_up and move it down (zero up and then right)
        moves, current = put_zero_below(current,x_up_f, y_up_f,layer,"left",n_down)
        path.extend(moves)
        
        moves, current = put_cuple_near(current)
        path.extend(moves)

        moves, current = put_cuple_right(current)
        path.extend(moves)
        
    last_one = len(FINAL_STATE)*(len(FINAL_STATE)-2)+ len(FINAL_STATE)   
    x_last, y_last = np.where(FINAL_STATE==last_one)
    print(last_one)

    moves,current = lead_n_to_pos(current,last_one,layer,x_last,y_last)
    path.extend(moves)
    return path,current


In [1886]:
path, _ = solve_last_two_layers(init_state)

[[ 1  2  3  4  5  6  7  8  9 10]
 [11 12 13 14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27 28 29 30]
 [31 32 33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48 49 50]
 [51 52 53 54 55 56 57 58 59 60]
 [61 62 63 64 65 66 67 68 69 70]
 [71 72 73 74 75 76 77 78 79 80]
 [99  0 87 96 81 89 86 85 93 97]
 [83 95 82 98 90 92 91 88 84 94]]
[[ 1  2  3  4  5  6  7  8  9 10]
 [11 12 13 14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27 28 29 30]
 [31 32 33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48 49 50]
 [51 52 53 54 55 56 57 58 59 60]
 [61 62 63 64 65 66 67 68 69 70]
 [71 72 73 74 75 76 77 78 79 80]
 [99 87  0 96 81 89 86 85 93 97]
 [83 95 82 98 90 92 91 88 84 94]]
[[ 1  2  3  4  5  6  7  8  9 10]
 [11 12 13 14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27 28 29 30]
 [31 32 33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48 49 50]
 [51 52 53 54 55 56 57 58 59 60]
 [61 62 63 64 65 66 67 68 69 70]
 [71 72 73 74 75 76 77 78 79 80]
 [99 87 96  0 81 89 86 85 93 97]
 [83 95 82 98 90 92 91 88 84 94]]
[[ 1  2

In [1887]:
print(path[-1])

[[ 1  2  3  4  5  6  7  8  9 10]
 [11 12 13 14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27 28 29 30]
 [31 32 33 34 35 36 37 38 39 40]
 [41 42 43 44 45 46 47 48 49 50]
 [51 52 53 54 55 56 57 58 59 60]
 [61 62 63 64 65 66 67 68 69 70]
 [71 72 73 74 75 76 77 78 79 80]
 [81 82 83 84 85 86 87 88 89 90]
 [91 92 93 94 95 96 97 98 99  0]]


## A-Star solver

#### Solve the last two lines with A-star

In [1888]:
def a_star_solver(initial_state: np.ndarray,layer) -> list[np.ndarray]:
    """Trova la soluzione del puzzle usando A*."""
    # Strutture dati per A*
    open_set = []
    heapq.heappush(open_set, (0, json.dumps(initial_state.tolist())))
    came_from = {}
    g_score = {json.dumps(initial_state.tolist()): 0}
    f_score = {json.dumps(initial_state.tolist()): manhattan_distance(initial_state)}

    visited = set()

    while open_set:
        _, current_str = heapq.heappop(open_set)
        current = np.array(json.loads(current_str))  # Convert string back to NumPy array
        visited.add(current_str)

        # Verifica se abbiamo trovato la soluzione
        if np.array_equal(current, FINAL_STATE):
            # Ricostruisci il percorso
            path = []
            while current_str in came_from:
                path.append(np.array(json.loads(current_str)))
                current_str = came_from[current_str]
            path.append(FINAL_STATE)
            return path[::-1]

        # Espandi i nodi
        for action in available_actions(current,layer):
            neighbor = do_action(current, action)
            neighbor_str = json.dumps(neighbor.tolist())

            if neighbor_str in visited:
                continue

            tentative_g_score = g_score[current_str] + 1

            if neighbor_str not in g_score or tentative_g_score < g_score[neighbor_str]:
                came_from[neighbor_str] = current_str
                g_score[neighbor_str] = tentative_g_score
                f_score[neighbor_str] = tentative_g_score + manhattan_distance(neighbor)
                heapq.heappush(open_set, (f_score[neighbor_str], neighbor_str))

    return None 

In [1889]:
'''print(curr_state)
moves = a_star_solver(curr_state,layer=len(FINAL_STATE)-2)
path.extend(moves)

print(f"Total moves: {len(path)}")
print("Final State:")
print(path[-1])'''

'print(curr_state)\nmoves = a_star_solver(curr_state,layer=len(FINAL_STATE)-2)\npath.extend(moves)\n\nprint(f"Total moves: {len(path)}")\nprint("Final State:")\nprint(path[-1])'